## **2D FDTD TM<sup>Z</sup> Solver**

Python adaptation of John B. Schneider's C program from section 8.4 of his textbook *Understanding the Finite-Difference Time-Domain
Method.*  This program simulates a TM<sup>Z</sup> 2D FDTD grid with a Ricker wavelet modeled as a hard source at the center of the grid.

Chapter 8 of Schneider's book is located [HERE](https://eecs.wsu.edu/~schneidj/ufdtd/chap8.pdf) and his GitHub source code is available [HERE](https://github.com/john-b-schneider/uFDTD).

In [ ]:
class oneDGrid: pass
class teZGrid: pass
class tmZGrid: pass
class threeDGrid: pass


class Grid:
    def __init__(self):
        self.Hx, self.Chxh, self.Chxe = None, None, None
        self.Hy, self.Chyh, self.Chye = None, None, None
        self.Hz, self.Chzh, self.Chze = None, None, None
        self.Ex, self.Cexe, self.Cexh = None, None, None
        self.Ey, self.Ceye, self.Ceyh = None, None, None
        self.Ez, self.Ceze, self.Cezh = None, None, None
        self.SizeX, self.SizeY, self.SizeZ = None, None, None
        self.Time, self.MaxTime = None, None
        self.Type = None
        self.Cdtds = None
        

def tmzdemo1():
    """TMz simulation with Ricker source at center of grid."""
    
    g = Grid()

    gridInit(g)        # initialize the grid
    ezIncInit(g)
    
    # do time stepping
    for g.Time in range(g.MaxTime):
        updateH2d(g)      # update magnetic field
        updateE2d(g)      # update electric field
        g.Ez[g.SizeX // 2, g.SizeY // 2] = ezInc(g.Time, 0.0); # add a source
        yield g.Ez, g.Hx, g.Hy

In [ ]:
import numpy as np

SIZEX = 101
SIZEY = 81
MAXTIME = 300

def gridInit(g):
    imp0 = 377.0
    g.Type = tmZGrid
    g.SizeX = SIZEX               # x size of domain
    g.SizeY = SIZEY               # y size of domain
    g.MaxTime = MAXTIME           # duration of simulation
    g.Cdtds = 1.0 / np.sqrt(2.0)  # Courant number

    g.Hx = np.zeros((g.SizeX, g.SizeY-1))
    g.Chxh = np.ones((g.SizeX, g.SizeY-1))
    g.Chxe = np.ones((g.SizeX, g.SizeY-1)) * g.Cdtds / imp0
    g.Hy = np.zeros((g.SizeX-1, g.SizeY))
    g.Chyh = np.ones((g.SizeX-1, g.SizeY))
    g.Chye = np.ones((g.SizeX-1, g.SizeY)) * g.Cdtds / imp0
    g.Ez = np.zeros((g.SizeX, g.SizeY))
    g.Ceze = np.ones((g.SizeX, g.SizeY))
    g.Cezh = np.ones((g.SizeX, g.SizeY)) * g.Cdtds * imp0

In [ ]:
import sys
cdtds = 0
ppw = 0


def ezIncInit(g):
    """Initialize source-function variables."""
    global cdtds, ppw
    cdtds = g.Cdtds
    ppw = 20


def ezInc(time, location):
    """Calculate source function at given time and location."""
    global cdtds, ppw
    if ppw <= 0:
        print("ezInc: ezIncInit() must be called before ezInc.\n"
              "       Points per wavelength must be positive.\n")
        sys.exit(-1)

    arg = np.pi * ((cdtds * time - location) / ppw - 1.0)
    arg = arg * arg

    return (1.0 - 2.0 * arg) * np.exp(-arg)

In [ ]:
def updateH2d(g):
    """Update magnetic field."""
    for mm in range(g.SizeX):
        for nn in range(g.SizeY-1):
            g.Hx[mm, nn] = g.Chxh[mm, nn] * g.Hx[mm, nn] \
            - g.Chxe[mm, nn] * (g.Ez[mm, nn + 1] - g.Ez[mm, nn])

    for mm in range(g.SizeX-1):
        for nn in range(g.SizeY):
            g.Hy[mm, nn] = g.Chyh[mm, nn] * g.Hy[mm, nn] \
            + g.Chye[mm, nn] * (g.Ez[mm + 1, nn] - g.Ez[mm, nn])


def updateE2d(g):
    """Update electric field."""
    for mm in range(1, g.SizeX-1):
        for nn in range(1, g.SizeY-1):
            g.Ez[mm, nn] = g.Ceze[mm, nn] * g.Ez[mm, nn] + \
            g.Cezh[mm, nn] * ((g.Hy[mm, nn] - g.Hy[mm - 1, nn]) - \
            (g.Hx[mm, nn] - g.Hx[mm, nn - 1]))

### Animation Setup

Functions to create animations of the 2D FDTD solvers.

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.animation as animation
from matplotlib import pyplot as plt, cm
from IPython import display
from IPython.display import HTML
from matplotlib import colors
plt.style.use('classic')
import numpy as np

# Set maximum animation file size
matplotlib.rcParams['animation.embed_limit'] = 40 * 2**20

# Defaults in raw2image.m
z_norm = 1 
decades = 3

# Create figure
fig, ax = plt.subplots(dpi=144)
im = ax.imshow(np.zeros((SIZEY, SIZEX)), cmap=cm.jet) # Transpose matrix 
plt.colorbar(im, ax=ax)
im.set_clim(-decades, 0)

def log_norm(data):
    """Return log normalized matrix."""
    return np.log10(np.abs((data)/z_norm)+np.nextafter(0, 1))
    
    
def anim_init():
    """Initialize plot for the animation."""
    im.set_data(np.zeros((SIZEY, SIZEX)))
    return (im,)


def animate(*args):
    """Draw the E-field and H-field magnitudes at current time step."""
    Ez, Hx, Hy = next(sim_step)
    data = Ez.transpose()
    data = log_norm(data)
    im.set_data(data)
    return (im,)


def html_video(frames):
    """Jupyter notebook must have animation converted to HTML to display."""
    mpl_animation = animation.FuncAnimation(
        fig, animate, frames=frames, init_func=anim_init, interval=25,
        blit=True
    )
    return mpl_animation


In [ ]:
sim_step = tmzdemo1()

anim = html_video(MAXTIME)
HTML(anim.to_jshtml())